In [14]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np
from sklearn.model_selection import train_test_split

print("Using TensorFlow version:", tf.__version__)

# Pour reproduire un comportement stable (optionnel)
tf.random.set_seed(42)

Using TensorFlow version: 2.18.0


## 1. Chargement du dataset

In [ ]:
# # Chemin vers le dossier qui contient les deux sous-dossiers : 'good' et 'defect'
# DATA_DIR = "screw_dataset"  # À adapter

# BATCH_SIZE = 32
# IMG_SIZE = (224, 224)

# # Charger toutes les images d'un seul tenant
# dataset = image_dataset_from_directory(
#     DATA_DIR,
#     labels='inferred',
#     label_mode='categorical',  # Pour un problème multi-classes, ici on suppose 2 classes (good, defect)
#     batch_size=BATCH_SIZE,
#     image_size=IMG_SIZE,
#     shuffle=True  # Mélange les données
# )

# # Afficher les classes détectées
# class_names = dataset.class_names
# print("Classes détectées :", class_names)

# # Optionnel : Normalisation et mise en cache
# AUTOTUNE = tf.data.AUTOTUNE
# def preprocess(image, label):
#     image = tf.cast(image, tf.float32) / 255.0  # Normalisation simple
#     return image, label

# dataset = dataset.map(preprocess).cache().prefetch(buffer_size=AUTOTUNE)

Found 1152 files belonging to 2 classes.
Classes détectées : ['bad', 'good']


In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np
from sklearn.model_selection import train_test_split
import os

# Chemin vers le dossier contenant 'good' et 'bad'
DATA_DIR = "screw_dataset"
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

# Obtenir la liste des noms de fichiers et leurs dossiers d'origine
file_paths = []
folders = []

for class_name in ['bad', 'good']:
    class_dir = os.path.join(DATA_DIR, class_name)
    for img_name in os.listdir(class_dir):
        if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            file_paths.append(os.path.join(class_dir, img_name))
            folders.append(class_name)

# Chargement des images
print("Chargement des images...")
dataset = image_dataset_from_directory(
    DATA_DIR,
    labels='inferred',
    label_mode='binary',  # Classification binaire
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False
)

# Extraction des images et labels
all_images = []
all_labels = []
for images, labels in dataset:
    all_images.append(images.numpy())
    all_labels.append(labels.numpy())

X = np.concatenate(all_images, axis=0) / 255.0  # Normalisation 
y = np.concatenate(all_labels, axis=0)

# Affichage des statistiques de base
print(f"Nombre total d'images: {X.shape[0]}")
print(f"Images 'bad' (label 0): {np.sum(y == 0)}")
print(f"Images 'good' (label 1): {np.sum(y == 1)}")

# Mélange des données
indices = np.arange(X.shape[0])
np.random.seed(42)
np.random.shuffle(indices)
X_shuffled = X[indices]
y_shuffled = y[indices]

# On mélange aussi les chemins de fichiers de la même façon
file_paths = np.array(file_paths)
folders = np.array(folders)
file_paths_shuffled = file_paths[indices]
folders_shuffled = folders[indices]

# Affichage des 10 premiers échantillons après mélange avec nom de fichier et dossier
print("\nPremiers échantillons après mélange:")
print("| N° | Nom de l'image                | Dossier d'origine | Label |")
print("|----|--------------------------------|-------------------|-------|")
for i in range(10):
    img_name = os.path.basename(file_paths_shuffled[i])
    folder = folders_shuffled[i]
    label = int(y_shuffled[i])
    print(f"| {i+1:2d} | {img_name:30s} | {folder:15s} | {label}     |")

# Division train/validation/test (70/15/15)
X_train, X_temp, y_train, y_temp, paths_train, paths_temp, folders_train, folders_temp = train_test_split(
    X_shuffled, y_shuffled, file_paths_shuffled, folders_shuffled, train_size=0.7, stratify=y_shuffled, random_state=42
)

X_val, X_test, y_val, y_test, paths_val, paths_test, folders_val, folders_test = train_test_split(
    X_temp, y_temp, paths_temp, folders_temp, train_size=0.5, stratify=y_temp, random_state=42
)

# Statistiques des ensembles
print(f"\nEnsemble d'entraînement: {X_train.shape[0]} images")
print(f"  - Bad (0): {np.sum(y_train == 0)} ({np.mean(y_train == 0)*100:.1f}%)")
print(f"  - Good (1): {np.sum(y_train == 1)} ({np.mean(y_train == 1)*100:.1f}%)")

print(f"\nEnsemble de validation: {X_val.shape[0]} images")
print(f"  - Bad (0): {np.sum(y_val == 0)} ({np.mean(y_val == 0)*100:.1f}%)")
print(f"  - Good (1): {np.sum(y_val == 1)} ({np.mean(y_val == 1)*100:.1f}%)")

print(f"\nEnsemble de test: {X_test.shape[0]} images")
print(f"  - Bad (0): {np.sum(y_test == 0)} ({np.mean(y_test == 0)*100:.1f}%)")
print(f"  - Good (1): {np.sum(y_test == 1)} ({np.mean(y_test == 1)*100:.1f}%)")

# Création des datasets TensorFlow
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

print("\nLes datasets sont prêts à être utilisés pour l'entraînement!")

Chargement des images...
Found 1152 files belonging to 2 classes.
Nombre total d'images: 1152
Images 'bad' (label 0): 285
Images 'good' (label 1): 867

Premiers échantillons après mélange:
| N° | Nom de l'image                | Dossier d'origine | Label |
|----|--------------------------------|-------------------|-------|
|  1 | 020_png.rf.bc1d8a583bfdb217fac2f64731e9771e.jpg | bad             | 0     |
|  2 | 196_png.rf.7fe8531648ca4aada3540a77f0ed51f6.jpg | good            | 1     |
|  3 | 009_png.rf.4539849089ddf72abf868f1b866a0d09.jpg | good            | 1     |
|  4 | 006_png.rf.c13a364cc262b36d021c69ab2a8533dc.jpg | bad             | 0     |
|  5 | 010_png.rf.1b22cf6babd655ecd7b71eff26a6310b.jpg | good            | 1     |
|  6 | 010_png.rf.49e214452922e22b40fe37eae05704b0.jpg | bad             | 0     |
|  7 | 032_png.rf.f74158859e091821a14a89d777c86dfa.jpg | good            | 1     |
|  8 | 210_png.rf.2476a4242693cc1c0b22a415c208ff82.jpg | good            | 1     |
|  9 | 307_p

/var/folders/c4/gwh8dn4x50x2lbs830bg6dkc0000gn/T/ipykernel_25689/148269067.py:69: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(y_shuffled[i])



Ensemble d'entraînement: 806 images
  - Bad (0): 199 (24.7%)
  - Good (1): 607 (75.3%)

Ensemble de validation: 173 images
  - Bad (0): 43 (24.9%)
  - Good (1): 130 (75.1%)

Ensemble de test: 173 images
  - Bad (0): 43 (24.9%)
  - Good (1): 130 (75.1%)

Les datasets sont prêts à être utilisés pour l'entraînement!


## 2. Définition d'un modèle simple
On définit un modèle CNN

In [20]:
num_classes = len(class_names)

model = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['accuracy']
)

model.summary()

/Users/emmacoco/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 93312)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     5,972,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,977,250 (22.80 MB)

 Trainable params: 5,977,250 (22.80 MB)

 Non-trainable params: 0 (0.00 B)

## 3. Entraînement du modèle

In [ ]:
EPOCHS = 1000  # Nombre d'époques d'entraînement

history = model.fit(
    dataset,                 # Entraînement ET test sur le même dataset
    epochs=EPOCHS,
    verbose=1
)

## 4. Évaluation sur le même dataset

In [ ]:
loss, accuracy = model.evaluate(dataset, verbose=0)
print(f"Loss sur le dataset complet : {loss:.4f}")
print(f"Accuracy sur le dataset complet : {accuracy:.4f}")